In [ ]:
import time
import os

import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm

import chipwhisperer as cw
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
from chipwhisperer.tests.tools_for_tests import FIRMWARE_DIR
from chipwhisperer.capture.api.programmers import XMEGAProgrammer

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure 
output_notebook()

In [ ]:
#connect to chipwhisperer
scope = cw.scope()
target = cw.target(scope)

In [ ]:
# setup scope parameters
scope.gain.gain = 45
scope.adc.samples = 3000
scope.adc.offset = 1250
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 7370000
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "clkgen"

In [ ]:
%%capture 
%%bash
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=CW303

In [ ]:
# program the target
programmer = XMEGAProgrammer()
programmer.scope = scope
programmer._logging = None
programmer.find()
programmer.erase()
aes_firmware_dir = os.path.join(FIRMWARE_DIR, 'simpleserial-aes')
aes_hex = os.path.join(aes_firmware_dir, r"simpleserial-aes-CW303.hex")
programmer.program('../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-CW303.hex', memtype="flash", verify=True)
programmer.close()

In [ ]:
TOOLS="crosshair,pan,wheel_zoom"

x = np.arange(3000)
y = np.zeros(3000)

p = figure(tools=TOOLS)
trace_plot = p.line(x,y)

In [ ]:
# get and explicit handle to update
target_plot = show(p, notebook_handle=True)

#Capture Traces
ktp = AcqKeyTextPattern_Basic(target=target)

traces = []
textin = []
keys = []
N = 50  # Number of traces
target.init()

for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    last_trace = scope.getLastTrace()
    traces.append(last_trace)
    
    trace_plot.data_source.data['y'] = last_trace
    push_notebook(handle=target_plot)
    #time.sleep(0.1)

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

In [ ]:
print("done")